This notebook with call the necessary functions to pass a data frame to an LLM and have it make suggestions for the output:

In [ ]:
#Library import for loading df
from llm_data_checker import read_df


#incase of error with file cache uncomment below command 
#importlib.reload(llm_data_checker)

data = read_df("data_test/Uncleaned_DS_jobs.csv")

#data.head()

Run this cell to check basic stats of df:

In [ ]:
from llm_data_checker import df_checker

check_data = df_checker(data)


#write out the stats to external directory
with open("outputs/stats.txt", "w") as f:
    for section_name, section_value in check_data.items():
        # Write a header for this section
        f.write(f"===== {section_name} =====\n")
        
        # Write the actual data
        f.write(str(check_data))
        
        # Add spacing
        f.write("\n\n")

Anonymiser: 

Prompt builder:

In [ ]:
from pathlib import Path 

#point to directoy holding prompt files
framework_txt = Path("frameworks")

#points to the directory holding the df stats 
stats_txt = Path("outputs")

#read the actual files themselves
system_template = (framework_txt / "prompt.txt").read_text()


func_test_suite = (framework_txt / "func_test_suite.txt").read_text()
function_format = (framework_txt / "function_format.txt").read_text()
stats = (stats_txt / "stats.txt").read_text()
reasoning = (framework_txt / "reasoning.txt").read_text()


#build the actual prompt 

prompt = system_template.format(
    func_test_suite=func_test_suite,
    function_format=function_format,
    stats=stats,
    reasoning=reasoning,
)

(Path("debug_prompt.txt")).write_text(prompt, encoding="utf-8")

API call out 


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4.1",  # or whatever model you're targeting
    messages=[
        {
            "role": "system",
            "content": prompt
        },
        {
            "role": "user",
            "content": "Proceed."
        }
    ],
    temperature=0.0,
)
print(response.output_text)

Write out results from LLM: